# 양자AI 미분 가능 프로그래밍

# 1. 양자컴퓨팅과 양자인공지능의 원리

In [2]:
using Symbolics
using Printf
using Statistics

## One Qubit CIrcuit

## 1.1 양자비트 상태 표현과 측정

### 켓 벡터

In [5]:
e₀ = [1.; 0.]
e₁ = [0.; 1.]

2-element Vector{Float64}:
 0.0
 1.0

In [6]:
@printf("<0|0> = %.1f, <1|1> = %.1f", e₀' * e₀, e₁' * e₁)

<0|0> = 1.0, <1|1> = 1.0

In [7]:
@printf("<0|1> = %.1f, <0|1> = %.1f", e₀' * e₁, e₁' * e₀)

<0|1> = 0.0, <0|1> = 0.0

In [8]:
@variables α β
q_cat = α*e₀ + β*e₁ 

2-element Vector{Num}:
 α
 β

### 양자상태측정

In [6]:
function measure(qubit_state, N::Int = 1)
    m_array = zeros(Int, N)
    for n in 1:N
        if rand() < abs(qubit_state[1])^2
            m_array[n] = 0
        else
            m_array[n] = 1
        end
    end
    ifelse(N==1, m_array[1],  m_array)
end 

measure (generic function with 2 methods)

In [7]:
sprint_array(X) = join(map(x->@sprintf("%d", x), X), ", ")
@printf("M(|0>) = %d\n", measure([1.;0.]))
println("M(|0>) = [", sprint_array(measure([1.;0.], 10)), "]")
@printf("M(|1>) = %d\n", measure([0.;1.]))
println("M(|1>) = [", sprint_array(measure([0.;1.], 10)), "]")

M(|0>) = 0
M(|0>) = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
M(|1>) = 1
M(|1>) = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
println("M(|+>) = M([1,1]/sqrt(2)) = [", sprint_array(measure([1.;1.]/sqrt(2), 10)), "]")
println("M(|->) = M([1,-1]/sqrt(2)) = [", sprint_array(measure([1.;-1.]/sqrt(2), 10)), "]")

M(|+>) = M([1,1]/sqrt(2)) = [0, 0, 0, 0, 1, 1, 0, 1, 1, 1]
M(|->) = M([1,-1]/sqrt(2)) = [0, 1, 1, 1, 1, 0, 1, 0, 0, 0]


#### 평균 구하기  

In [9]:
N = 100
@printf("Average of %d times measurements of |0> is %.2f\n", 
    N, mean(measure([1.;0.], N)))
@printf("Average of %d times measurements of |1> is %.2f\n", 
    N, mean(measure([0.;1.], N)))
@printf("Average of %d times measurements of |+> is %.2f\n", 
    N, mean(measure([1.;1.]/sqrt(2), N)))
@printf("Average of %d times measurements of |-> is %.2f\n", 
    N, mean(measure([1.;-1.]/sqrt(2), N)))

Average of 100 times measurements of |0> is 0.00
Average of 100 times measurements of |1> is 1.00
Average of 100 times measurements of |+> is 0.57
Average of 100 times measurements of |-> is 0.50


## 1.2 단일 양자비트 연산자

### 양자 반전 연산자

In [10]:
X = e₁ * e₀' + e₀ * e₁'

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [24]:
println(X * e₀ == e₁, ", ", X * e₁ == e₀)

true, true


In [25]:
display(X * q_cat)

2-element Vector{Num}:
 β
 α

### 양자중첩 연산자

In [27]:
H = [1. 1.; 1. -1.] / sqrt(2)

2×2 Matrix{Float64}:
 0.707107   0.707107
 0.707107  -0.707107

In [31]:
e₊ = H * e₀
e₋ = H * e₁
e₊, e₋ 

([0.7071067811865475, 0.7071067811865475], [0.7071067811865475, -0.7071067811865475])

In [33]:
println(e₊ == (e₀ + e₁) / sqrt(2), ", ", e₋ == (e₀ - e₁) / sqrt(2))

true, true


In [37]:
H*q_cat * sqrt(2)

2-element Vector{Num}:
 α + β
 α - β

### 파울리 연산자

In [63]:
Y = [0 -1.0im; 1.0im 0]
Z = [1. 0; 0 -1.] 
display(Y)
display(Z)

2×2 Matrix{ComplexF64}:
 0.0+0.0im  -0.0-1.0im
 0.0+1.0im   0.0+0.0im

2×2 Matrix{Float64}:
 1.0   0.0
 0.0  -1.0

In [66]:
display(Y * q_cat)
display(Z * q_cat)

2-element Vector{Complex{Num}}:
 -im*β
  im*α

2-element Vector{Num}:
  α
 -β

## 1.3 다중 양자비트 표현 및 연산

### 다중 양자비트 상태 표현

In [112]:
@variables α[1:2] β[1:2]

Qubit1 = α[1] * e₀ + β[1] * e₁
Qubit2 = α[2] * e₀ + β[2] * e₁
Qubits = [Qubit1 Qubit2]

2×2 Matrix{Num}:
 α[1]  α[2]
 β[1]  β[2]

In [113]:
Qubit12_state = kron(Qubit1, Qubit2)

4-element Vector{Num}:
 α[1]*α[2]
 α[1]*β[2]
 α[2]*β[1]
 β[1]*β[2]

In [114]:
E1 = [e₀ e₁]
E2 = [kron(e₀, e₀) kron(e₀, e₁) kron(e₁, e₀) kron(e₁, e₁)] 
display(E1)
display(E2)
E2 == kron(E1, E1)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

true

### 양자 제어반전 연산자

In [115]:
CNOT = [1. 0 0 0; 0 1. 0 0; 0 0 0 1.; 0 0 1. 0]
display(CNOT)
display(CNOT * E2)
display(CNOT * Qubit12_state)

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0

4-element Vector{Num}:
 α[1]*α[2]
 α[1]*β[2]
 β[1]*β[2]
 α[2]*β[1]

In [116]:
function fn_cnot(q1, q2)
    CNOT = [1. 0 0 0; 0 1. 0 0; 0 0 0 1.; 0 0 1. 0]
    q12 = kron(q1, q2)
    return CNOT * q12
end

display(fn_cnot(e₁, e₀))
display(fn_cnot(Qubit1, Qubit2))

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0

4-element Vector{Num}:
 α[1]*α[2]
 α[1]*β[2]
 β[1]*β[2]
 α[2]*β[1]

In [117]:
function calc_new_q(new_state)
    f(tmp, d, idx) = expand(substitute(tmp[idx], d))

    tmp1 = [sqrt(sum(new_state[1:2].^2)); sqrt(sum(new_state[3:4].^2))]
    d1 = Dict([α[2]^2 => 1 - β[2]^2])     
    new_q1 = map(k -> f(tmp1, d1, k), [1,2])

    tmp2 = [sqrt(sum(new_state[1:2:4].^2)); sqrt(sum(new_state[2:2:4].^2))]
    d2 = Dict([α[1]^2 => 1 - β[1]^2])     
    new_q2 = map(k -> f(tmp2, d2, k), [1,2])
    
    [new_q1 new_q2]
end

calc_new_q (generic function with 1 method)

In [118]:
new_Q = calc_new_q(Qubit12_state)
display(new_Q[:,1])
display(new_Q[:,2])

2-element Vector{Num}:
 sqrt(α[1]^2)
 sqrt(β[1]^2)

2-element Vector{Num}:
 sqrt(α[2]^2)
 sqrt(β[2]^2)

In [119]:
new_state = CNOT * Qubit12_state
new_Q = calc_new_q(new_state)
display(new_Q[:,1])
display(new_Q[:,2])

2-element Vector{Num}:
 sqrt(α[1]^2)
 sqrt(β[1]^2)

2-element Vector{Num}:
 sqrt((β[1]^2)*(β[2]^2) + α[2]^2 - (α[2]^2)*(β[1]^2))
 sqrt((α[2]^2)*(β[1]^2) + β[2]^2 - (β[1]^2)*(β[2]^2))